In [1]:
import requests
import pandas as pd
from service import *
import os
from config.config import *
from web_scrape import *
from update import *
from extract import *
from transform import *

In [2]:
update_circuits_data()
update_drivers_data()
update_constructors_data()
update_races_data()
update_results_data()
update_laptime_data()
update_pitstops_data()
update_qualifying_data()

there are 76 rows in circuits table in database
there are 77 rows in circuits data in Ergast


c:\Users\B RU CE\Downloads\LaFormula\transform.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['length'] = data['url'].apply(get_length_data_scrape)
c:\Users\B RU CE\Downloads\LaFormula\transform.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['turns'] = data['url'].apply(get_turns_data_scrape)
c:\Users\B RU CE\Downloads\LaFormula\transform.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

there are 860 rows in drivers table in database
there are 860 rows in drivers data in Ergast
there are 212 rows in constructors table in database
there are 212 rows in constructors data in Ergast
there are 1125 rows in races table in database
there are 1125 rows in races data in Ergast
there are 26599 rows in races table in database
there are 26619 rows in races data in Ergast
start updating values from api offset 26599
start updating laptime data from season 2024 round 16 to the latest in results table
starting pulling laptime
start updating pitstops data from season 2024 round 17 to the latest in results table
start updating qualifying data from season 2024 round 17 to the latest in results table


In [12]:
def get_laptime_data(starting_season=1996,starting_round=1,offset=0):
    results = pd.DataFrame(columns=['position','time','season','round','lap','driverId'])
    for season,round in get_years_rounds(starting_season,starting_round):
        total = int(get_api_json(laptimes_api.format(year=str(season),round=str(round)),params={'limit':1})['MRData']['total'])
        while offset <= total:
            raw = get_api_json(laptimes_api.format(year=str(season),round=str(round)),params={'limit':100,'offset': offset})
            for laps in raw['MRData']['RaceTable']['Races'][0]['Laps']:
                timing_table = pd.DataFrame(laps['Timings'])
                timing_table[['season','round','lap']] = season,round,int(laps['number'])
                results = pd.concat([results,timing_table],axis=0,ignore_index=True)
            offset += 100
    return results

In [14]:
test = get_laptime_data(2024,16)

In [15]:
test

,position,time,season,round,lap,driverId
0,1,1:27.623,2024,16,1,piastri
1,2,1:28.179,2024,16,1,leclerc
2,3,1:28.971,2024,16,1,norris
3,4,1:29.345,2024,16,1,sainz
4,5,1:29.674,2024,16,1,hamilton
...,...,...,...,...,...,...
1003,9,1:23.715,2024,16,53,kevin_magnussen
1004,10,1:23.918,2024,16,53,albon
1005,11,1:23.686,2024,16,53,alonso
1006,12,1:24.347,2024,16,53,ricciardo


In [11]:
get_api_json(laptimes_api.format(year=str(2020),round=str(1))['MRData']['total'])

TypeError: get_api_json() missing 1 required positional argument: 'params'

In [6]:
test

(2024, 16)

In [5]:
total = int(get_api_json(pitstops_api.format(year=2024,round=2),params={'limit':1})['MRData']['total'])

In [2]:
def get_api_json(url,params):
    response = requests.get(url, params=params)
    raw_data = response.json()
    return raw_data

In [3]:
def get_circuits_data(offset=0):
    total = int(get_api_json(circuits_api,params={'limit':1})['MRData']['total'])
    results = pd.DataFrame(columns=['circuitId','url','circuitName','Location'])
    while offset <= total:
        raw = get_api_json(circuits_api,{'limit': 1000,'offset':offset})
        circuits = raw['MRData']['CircuitTable']['Circuits']
        data = pd.DataFrame(circuits)
        results = pd.concat([results,data],ignore_index=True)
        offset += 1000
    return results

In [4]:
raw = get_circuits_data(offset=0)

In [5]:
raw

,circuitId,url,circuitName,Location
0,adelaide,http://en.wikipedia.org/wiki/Adelaide_Street_C...,Adelaide Street Circuit,"{'lat': '-34.9272', 'long': '138.617', 'locali..."
1,ain-diab,http://en.wikipedia.org/wiki/Ain-Diab_Circuit,Ain Diab,"{'lat': '33.5786', 'long': '-7.6875', 'localit..."
2,aintree,http://en.wikipedia.org/wiki/Aintree_Motor_Rac...,Aintree,"{'lat': '53.4769', 'long': '-2.94056', 'locali..."
3,albert_park,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,Albert Park Grand Prix Circuit,"{'lat': '-37.8497', 'long': '144.968', 'locali..."
4,americas,http://en.wikipedia.org/wiki/Circuit_of_the_Am...,Circuit of the Americas,"{'lat': '30.1328', 'long': '-97.6411', 'locali..."
...,...,...,...,...
72,yas_marina,http://en.wikipedia.org/wiki/Yas_Marina_Circuit,Yas Marina Circuit,"{'lat': '24.4672', 'long': '54.6031', 'localit..."
73,yeongam,http://en.wikipedia.org/wiki/Korean_Internatio...,Korean International Circuit,"{'lat': '34.7333', 'long': '126.417', 'localit..."
74,zandvoort,http://en.wikipedia.org/wiki/Circuit_Zandvoort,Circuit Park Zandvoort,"{'lat': '52.3888', 'long': '4.54092', 'localit..."
75,zeltweg,http://en.wikipedia.org/wiki/Zeltweg_Airfield,Zeltweg,"{'lat': '47.2039', 'long': '14.7478', 'localit..."


In [6]:
raw2= clean_circuits_data(raw)

c:\Users\B RU CE\Downloads\LaFormula\transform.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['length'] = data['url'].apply(get_length_data_scrape)
c:\Users\B RU CE\Downloads\LaFormula\transform.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['turns'] = data['url'].apply(get_turns_data_scrape)
c:\Users\B RU CE\Downloads\LaFormula\transform.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [1]:
from load import *

In [2]:
load_circuits_data_to_db()

c:\Users\B RU CE\Downloads\LaFormula\transform.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['length'] = data['url'].apply(get_length_data_scrape)
c:\Users\B RU CE\Downloads\LaFormula\transform.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['turns'] = data['url'].apply(get_turns_data_scrape)
c:\Users\B RU CE\Downloads\LaFormula\transform.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in